In [ ]:
import matplotlib.pyplot as plt

def plot_results(axes, results_df, dataset, learner):
    
    axes.scatter(results_df["PREDICTION_SCORE"],
                 results_df["ASSAY_SCORE"], color=np.random.rand(3,))
    axes.set_title(f'{dataset}/{learner}')
    axes.set_ylabel('Assay Value')
    axes.set_xlabel('Prediction')


In [ ]:
import context  # noqa: F401
from msalde.container import ALDEContainer

datasets = ["cas12f"]
container = ALDEContainer()
repo = container.query_repository

num_rows = (len(datasets) * 3) // 3 + (len(datasets) % 3 > 0)
fig, axes = plt.subplots(num_rows, 3, figsize=(15, 8))
axes = axes.flatten()

for i, dataset in enumerate(datasets):
    rf_results = repo.get_top_variants_by_dataset_learner(
        dataset_name=dataset, learner_name="RF_AL")
    ind = i * 3
    plot_results(axes[ind], rf_results, dataset, "RandomForest AL")
    rf_results = repo.get_top_variants_by_dataset_learner(
        dataset_name=dataset, learner_name="RF_TRAINALL")
    ind += 1
    plot_results(axes[ind], rf_results, dataset, "RandomForest TrainAll")
    llr_results = repo.get_top_variants_by_dataset_learner(
        dataset_name=dataset, learner_name="ESM2LogLikelihood")
    ind += 1
    plot_results(axes[ind], rf_results, dataset, "LogLikelihood")

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()
